После обновления баланса в игре, где сражаются 4 юнита в бою против мобов, изменилось время боя.

In [166]:
import pandas as pd
import matplotlib as plt
import seaborn as sns
import plotly.express as px
import numpy as np
df=pd.read_csv('vk.csv', encoding='mac_cyrillic')

In [2]:
#сразу режем датасет на 2
df_bef = df.query('balance_mark == "before"')
df_aft = df.query('balance_mark == "after"')
#проверяем

In [3]:
df_aft

,battle_id,unit_1,unit_2,unit_3,unit_4,len,balance_mark
10000,103978,меч,булава,меч,булава,23.30,after
10001,108601,посох,топор,булава,топор,27.12,after
10002,112054,лук,булава,лук,лук,26.06,after
10003,119433,посох,посох,лук,булава,22.76,after
10004,119938,меч,посох,топор,посох,28.40,after
...,...,...,...,...,...,...,...
19995,108834,посох,булава,лук,посох,26.37,after
19996,119941,посох,меч,лук,посох,23.73,after
19997,112911,топор,посох,булава,топор,27.73,after
19998,111684,меч,топор,посох,меч,24.20,after


In [4]:
df_bef

,battle_id,unit_1,unit_2,unit_3,unit_4,len,balance_mark
0,103921,топор,лук,булава,лук,17.35,before
1,117214,посох,посох,лук,лук,21.84,before
2,115502,меч,посох,булава,топор,16.00,before
3,110102,булава,лук,лук,булава,17.22,before
4,104989,меч,посох,булава,посох,19.20,before
...,...,...,...,...,...,...,...
9995,110026,посох,лук,лук,булава,17.88,before
9996,113821,топор,лук,посох,лук,16.26,before
9997,105155,меч,лук,посох,меч,20.58,before
9998,104050,лук,меч,булава,посох,20.08,before


In [5]:
#удаляем неиформативный столбец
df_aft = df_aft.drop(['balance_mark'], axis = 1)
df_bef = df_bef.drop(['balance_mark'], axis = 1)

In [14]:
#1 слой группировки. Сразу вычисляем среднее по аналогичнм наборам, где порядок пока учитывается
#Слабое место в решении
df_aft_new=df_aft.groupby(['unit_1', 'unit_2', 'unit_3', 'unit_4'], as_index = False).agg({'len':'mean'})
df_bef_new=df_bef.groupby(['unit_1', 'unit_2', 'unit_3', 'unit_4'], as_index = False).agg({'len':'mean'})

In [54]:
# формируем датасет из двух, где будет сравнение времени
df_all = df_bef_new
df_all = df_all.rename(columns={'len': 'len_before'})
df_all['len_after'] = df_aft_new.len
df_all.head(6)

,unit_1,unit_2,unit_3,unit_4,len_before,len_after
0,булава,булава,булава,булава,18.518125,19.811667
1,булава,булава,булава,лук,21.400588,24.562381
2,булава,булава,булава,меч,21.597273,20.456667
3,булава,булава,булава,посох,19.918947,25.261053
4,булава,булава,булава,топор,19.117778,18.867500
5,булава,булава,лук,булава,19.898182,25.669375


In [95]:
df_all.shape

(625, 6)

* Видно что в строке 1 и 5 по факту один и тот же набор.  
* Всего строчек в нашем датасете 625, а формула сочетания с повторениями даёт значение 70 различных комбинаций (5 видов оружия по 4 слотам)

In [96]:
#создаём колонки для сводной таблицы
df_last = df_all.assign(bul_count = 0, bow_count = 0, sword_count = 0, axe_count = 0, pos_count = 0)
df_last

,unit_1,unit_2,unit_3,unit_4,len_before,len_after,bul_count,bow_count,sword_count,axe_count,pos_count
0,булава,булава,булава,булава,18.518125,19.811667,0,0,0,0,0
1,булава,булава,булава,лук,21.400588,24.562381,0,0,0,0,0
2,булава,булава,булава,меч,21.597273,20.456667,0,0,0,0,0
3,булава,булава,булава,посох,19.918947,25.261053,0,0,0,0,0
4,булава,булава,булава,топор,19.117778,18.867500,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
620,топор,топор,топор,булава,19.184375,19.642727,0,0,0,0,0
621,топор,топор,топор,лук,20.427826,25.535556,0,0,0,0,0
622,топор,топор,топор,меч,20.541500,21.120833,0,0,0,0,0
623,топор,топор,топор,посох,20.475500,24.838462,0,0,0,0,0


#### формируем и заполняем таблицу с помощью цикла

In [93]:
d = {'1' : "unit_1", '2' : "unit_2" , '3' : "unit_3", '4' : "unit_4"}
df_last.bul_count = 0
d.get('1')
for j in range(len(d)):
    for i in range(len(df_last)):
        if df_last.loc[i, d.get(str(j+1))] == "булава":
            df_last.loc[i, 'bul_count']+=1
        if df_last.loc[i, d.get(str(j+1))] == "меч":
            df_last.loc[i, 'sword_count']+=1
        if df_last.loc[i, d.get(str(j+1))] == "лук":
            df_last.loc[i, 'bow_count']+=1    
        if df_last.loc[i, d.get(str(j+1))] == "посох":
            df_last.loc[i, 'pos_count']+=1
        if df_last.loc[i, d.get(str(j+1))] == "топор":
            df_last.loc[i, 'axe_count']+=1
# удаляем неинформативные колонки
df_last = df_last.drop(['unit_1', 'unit_2','unit_3','unit_4'], axis = 1)
df_last

,len_before,len_after,bul_count,bow_count,sword_count,axe_count,pos_count
0,18.518125,19.811667,4,0,0,0,0
1,21.400588,24.562381,3,1,0,0,0
2,21.597273,20.456667,3,0,1,0,0
3,19.918947,25.261053,3,0,0,0,1
4,19.117778,18.867500,3,0,0,1,0
...,...,...,...,...,...,...,...
620,19.184375,19.642727,1,0,0,3,0
621,20.427826,25.535556,0,1,0,3,0
622,20.541500,21.120833,0,0,1,3,0
623,20.475500,24.838462,0,0,0,3,1


In [45]:
#2 слой групировки
fin_df = df_last.groupby(['bul_count', 
                          'bow_count', 
                          'sword_count', 
                          'axe_count', 
                          'pos_count'], 
                         as_index = False) \
                .agg({'len_after'  : 'mean', 
                      'len_before' : 'mean' })
fin_df

,bul_count,bow_count,sword_count,axe_count,pos_count,len_after,len_before
0,0,0,0,0,4,24.978182,20.571250
1,0,0,0,1,3,24.730281,19.896264
2,0,0,0,2,2,25.074832,20.164788
3,0,0,0,3,1,24.870167,19.982164
4,0,0,0,4,0,19.969167,18.891667
...,...,...,...,...,...,...,...
65,3,0,0,0,1,25.531430,19.531053
66,3,0,0,1,0,19.902857,20.160688
67,3,0,1,0,0,20.023058,20.223728
68,3,1,0,0,0,24.963388,20.379802


### видим как раз 70 комбинаций, значит сгрупировали правильно и в некотором приближении решили правильно

In [134]:
fin_df['time_diff'] = fin_df.len_after - fin_df.len_before
fig = px.histogram(fin_df, x = 'time_diff') 
fig.show()

### * В целом видно, что большая часть оружия, точнее комбинации оружия стали слабее def>3.5. 
### * Небольшая часть комбинаций стала слегка слабее
### * И есть один набор который сильно уменьшил время в бою после обновлении(возможно выброс)

In [148]:
dif_df = fin_df.drop(columns = ['len_after', 'len_before'])
dif_df.query('time_diff < -1') 

,bul_count,bow_count,sword_count,axe_count,pos_count,time_diff
59,2,0,1,1,0,-10.064594


#### две булавы, топор и меч оказалось плохой комбинацией, посмотрим на схожие наборы

In [158]:
dif_df.query('bul_count == 2 and axe_count == 1 and sword_count != 1') 

,bul_count,bow_count,sword_count,axe_count,pos_count,time_diff
56,2,0,0,1,1,4.557384
62,2,1,0,1,0,5.392586


In [159]:
dif_df.query('bul_count == 2 and axe_count != 1 and sword_count == 1') 

,bul_count,bow_count,sword_count,axe_count,pos_count,time_diff
58,2,0,1,0,1,5.600644
63,2,1,1,0,0,4.397300


In [161]:
dif_df.query('bul_count in [1,3] and axe_count == 1 and sword_count == 1') 

,bul_count,bow_count,sword_count,axe_count,pos_count,time_diff
40,1,0,1,1,1,4.906408
49,1,1,1,1,0,4.754011


### Все похожие наборы (отличающиеся от исходного на 1 оружие), наоборот имеют заметное увеличения времени боя, значит это скорее всего выброс, либо какая-то безумная синергия. Но лучше это выброс убрать

In [229]:
dif_df = dif_df.drop(59)

In [230]:
dif_df.query('time_diff>5.5').describe()

,bul_count,bow_count,sword_count,axe_count,pos_count,time_diff
count,6.000000,6.000000,6.000000,6.00000,6.000000,6.000000
mean,0.833333,0.333333,0.833333,0.50000,1.500000,5.684495
std,1.329160,0.516398,0.752773,0.83666,1.048809,0.176930
min,0.000000,0.000000,0.000000,0.00000,0.000000,5.527352
25%,0.000000,0.000000,0.250000,0.00000,1.000000,5.554483
50%,0.000000,0.000000,1.000000,0.00000,1.500000,5.647023
75%,1.500000,0.750000,1.000000,0.75000,2.000000,5.732924
max,3.000000,1.000000,2.000000,2.00000,3.000000,6.000377


In [231]:
dif_df.query('time_diff>5.5')

,bul_count,bow_count,sword_count,axe_count,pos_count,time_diff
6,0,0,1,1,2,5.539096
9,0,0,2,0,2,5.746098
15,0,1,0,0,3,5.693402
21,0,1,1,2,0,5.527352
58,2,0,1,0,1,5.600644
65,3,0,0,0,1,6.000377


In [265]:
dif_df.query('pos_count > 1').time_diff.mean()

5.068836228267768

In [266]:
dif_df.query('axe_count > 1').time_diff.mean()

3.191394174858108

In [267]:
dif_df.query('sword_count > 1').time_diff.mean()

3.0507231527578087

In [268]:
dif_df.query('bow_count > 1').time_diff.mean()

4.782337015305762

In [269]:
dif_df.query('bul_count > 1').time_diff.mean()

3.2752364889880616

### Видно, что самым пострадавшим оружием в среднем является "посох"

In [282]:
dif_df.query('-1<time_diff<2.6').describe()

,bul_count,bow_count,sword_count,axe_count,pos_count,time_diff
count,14.000000,14.0,14.000000,14.000000,14.0,14.000000
mean,1.285714,0.0,1.357143,1.357143,0.0,0.319702
std,1.325987,0.0,1.336306,1.336306,0.0,0.658931
min,0.000000,0.0,0.000000,0.000000,0.0,-0.303326
25%,0.000000,0.0,0.000000,0.000000,0.0,-0.184747
50%,1.000000,0.0,1.000000,1.000000,0.0,0.104432
75%,2.000000,0.0,2.000000,2.000000,0.0,0.474583
max,4.000000,0.0,4.000000,4.000000,0.0,1.840300


In [283]:
dif_df.query('-1<time_diff<2.5')

,bul_count,bow_count,sword_count,axe_count,pos_count,time_diff
4,0,0,0,4,0,1.077500
8,0,0,1,3,0,0.496032
11,0,0,2,2,0,0.410236
13,0,0,3,1,0,-0.067441
14,0,0,4,0,0,1.840300
38,1,0,0,3,0,-0.303326
41,1,0,1,2,0,-0.020992
43,1,0,2,1,0,0.338936
44,1,0,3,0,0,-0.136974
57,2,0,0,2,0,0.229855


### Очевидно, теперь ещё два факта:
* Почти на ровне с посохом пострадал и лук, так как в наборах, чьё время почти не изменилось их нет совсем
* Меч и Топор хоть и немного отличась от булавы, показвают себя одинаково хорошо и пострадали меньше при обновлении

*здесь уместо вспмнить, что как раз именно меч топор и две булавы показали сверх хороший результат*